In [ ]:
# !pip install tensorflow numpy

In [1]:
import os
import warnings
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step




Create a simple neural network model with a Flatten layer followed by two Dense layers.


In [2]:
# Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])


Define Loss Function and Optimizer:

- Use Sparse Categorical Crossentropy for the loss function.
- Use the Adam optimizer.


In [3]:
# Define Loss Function and Optimizer

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()


Implement the Custom Training Loop:

- Iterate over the dataset for a specified number of epochs.
- Compute the loss and apply gradients to update the model's weights.


In [4]:
# Implement the Custom Training Loop

epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')


Start of epoch 1
Epoch 1 Step 0: Loss = 2.342745304107666
Epoch 1 Step 200: Loss = 0.4407643973827362
Epoch 1 Step 400: Loss = 0.16888663172721863
Epoch 1 Step 600: Loss = 0.1717824637889862
Epoch 1 Step 800: Loss = 0.1786532998085022
Epoch 1 Step 1000: Loss = 0.4041965901851654
Epoch 1 Step 1200: Loss = 0.2196640968322754
Epoch 1 Step 1400: Loss = 0.25246381759643555
Epoch 1 Step 1600: Loss = 0.229383185505867
Epoch 1 Step 1800: Loss = 0.18786725401878357
Start of epoch 2
Epoch 2 Step 0: Loss = 0.07759848237037659
Epoch 2 Step 200: Loss = 0.21298855543136597
Epoch 2 Step 400: Loss = 0.12948989868164062
Epoch 2 Step 600: Loss = 0.037624541670084
Epoch 2 Step 800: Loss = 0.11023370921611786
Epoch 2 Step 1000: Loss = 0.1761862188577652
Epoch 2 Step 1200: Loss = 0.13388654589653015
Epoch 2 Step 1400: Loss = 0.16112855076789856
Epoch 2 Step 1600: Loss = 0.17073768377304077
Epoch 2 Step 1800: Loss = 0.09309744089841843


Adding Accuracy Metric:

Enhance the custom training loop by adding an accuracy metric to monitor model performance.





In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


Define the Model:
Use the same model as earlier


In [6]:
#  Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])


Define the loss function, optimizer, and metric:

- Use Sparse Categorical Crossentropy for the loss function and Adam optimizer.

- Add Sparse Categorical Accuracy as a metric.


In [7]:
#  Define Loss Function, Optimizer, and Metric

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


Implement the custom training loop with accuracy:

Track the accuracy during training and print it at regular intervals.


In [8]:
# Implement the Custom Training Loop with Accuracy

epochs = 5  # Number of epochs for training

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)

        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')

    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()


Start of epoch 1
Epoch 1 Step 0: Loss = 2.3675007820129395 Accuracy = 0.0625
Epoch 1 Step 200: Loss = 0.40460026264190674 Accuracy = 0.832866907119751
Epoch 1 Step 400: Loss = 0.19316300749778748 Accuracy = 0.8666614890098572
Epoch 1 Step 600: Loss = 0.16448861360549927 Accuracy = 0.8831114768981934
Epoch 1 Step 800: Loss = 0.15014925599098206 Accuracy = 0.8952481150627136
Epoch 1 Step 1000: Loss = 0.4227451682090759 Accuracy = 0.9023164510726929
Epoch 1 Step 1200: Loss = 0.17295680940151215 Accuracy = 0.9090081453323364
Epoch 1 Step 1400: Loss = 0.23636934161186218 Accuracy = 0.9143692255020142
Epoch 1 Step 1600: Loss = 0.19532684981822968 Accuracy = 0.9173368215560913
Epoch 1 Step 1800: Loss = 0.1667139232158661 Accuracy = 0.9212590456008911
Start of epoch 2
Epoch 2 Step 0: Loss = 0.1011013612151146 Accuracy = 0.96875
Epoch 2 Step 200: Loss = 0.15126866102218628 Accuracy = 0.9619091749191284
Epoch 2 Step 400: Loss = 0.09804297983646393 Accuracy = 0.9586970210075378
Epoch 2 Step 600: 

Custom Callback for Advanced Logging:

Implement a custom callback to log additional metrics and information during training.



In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


Define the Model:

Use the same model as earlier


In [10]:
#  Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])


Define Loss Function, Optimizer, and Metric:

- Use Sparse Categorical Crossentropy for the loss function and Adam optimizer.

- Add Sparse Categorical Accuracy as a metric.


In [11]:
# Define Loss Function, Optimizer, and Metric

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


Implement the custom training loop with custom callback:

Create a custom callback to log additional metrics at the end of each epoch.


In [12]:
from tensorflow.keras.callbacks import Callback

# Implement the Custom Callback
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}')


In [13]:
# Implement the Custom Training Loop with Custom Callback

epochs = 2
custom_callback = CustomCallback()  # Initialize the custom callback

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)

        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')

    # Call the custom callback at the end of each epoch
    custom_callback.on_epoch_end(epoch, logs={'loss': loss_value.numpy(), 'accuracy': accuracy_metric.result().numpy()})

    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()  # Use reset_state() instead of reset_states()


Start of epoch 1
Epoch 1 Step 0: Loss = 2.3673155307769775 Accuracy = 0.09375
Epoch 1 Step 200: Loss = 0.34604182839393616 Accuracy = 0.832866907119751
Epoch 1 Step 400: Loss = 0.19698874652385712 Accuracy = 0.8679083585739136
Epoch 1 Step 600: Loss = 0.16240321099758148 Accuracy = 0.8839954137802124
Epoch 1 Step 800: Loss = 0.1640537977218628 Accuracy = 0.8972378373146057
Epoch 1 Step 1000: Loss = 0.44047534465789795 Accuracy = 0.9045642018318176
Epoch 1 Step 1200: Loss = 0.1672089397907257 Accuracy = 0.9111157655715942
Epoch 1 Step 1400: Loss = 0.2854663133621216 Accuracy = 0.915752112865448
Epoch 1 Step 1600: Loss = 0.22955010831356049 Accuracy = 0.9190349578857422
Epoch 1 Step 1800: Loss = 0.19465573132038116 Accuracy = 0.9230288863182068
End of epoch 1, loss: 0.031704697757959366, accuracy: 0.9248666763305664
Start of epoch 2
Epoch 2 Step 0: Loss = 0.07026565819978714 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.1476096659898758 Accuracy = 0.9625310897827148
Epoch 2 Step 400: Loss = 

Implement the custom training loop with custom callback:

Create a custom callback to log additional metrics at the end of each epoch.


In [14]:
from tensorflow.keras.layers import Input, Dense

# Define the input layer
input_layer = Input(shape=(28, 28))  # Input layer with shape (28, 28)

# Define hidden layers
hidden_layer1 = Dense(64, activation='relu')(input_layer)  # First hidden layer with 64 neurons and ReLU activation
hidden_layer2 = Dense(64, activation='relu')(hidden_layer1)  # Second hidden layer with 64 neurons and ReLU activation


In the above code:

`Dense(64, activation='relu')` creates a dense (fully connected) layer with 64 units and ReLU activation function.

Each hidden layer takes the output of the previous layer as its input.


Define the output layer

define the output layer. Suppose you are working on a binary classification problem, so the output layer will have one unit with a sigmoid activation function.


In [15]:
output_layer = Dense(1, activation='sigmoid')(hidden_layer2)

In the above code:

`Dense(1, activation='sigmoid')` creates a dense layer with 1 unit and a sigmoid activation function, suitable for binary classification.


Create the Model <br/>

create the model by specifying the input and output layers.


In [ ]:
model = Model(inputs=input_layer, outputs=output_layer)

In the above code:

`Model(inputs=input_layer, outputs=output_layer)` creates a Keras model that connects the input layer to the output layer through the hidden layers.


Compile the Model

Before training the model, it needs to be compiled. specify the loss function, optimizer, and evaluation metrics.


In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In the above code:

`optimizer='adam'` specifies the Adam optimizer, a popular choice for training neural networks.

`loss='binary_crossentropy'` specifies the loss function for binary classification problems.

`metrics=['accuracy']` tells Keras to evaluate the model using accuracy during training.


Train the Model

train the model on some training data.  `X_train` is our training input data and `y_train` is the corresponding labels.


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import numpy as np

#  Redefine the Model for 20 features
model = Sequential([
    Input(shape=(20,)),  # Adjust input shape to (20,)
    Dense(128, activation='relu'),  # Hidden layer with 128 neurons and ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification with sigmoid activation
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#  Generate Example Data
X_train = np.random.rand(1000, 20)  # 1000 samples, 20 features each
y_train = np.random.randint(2, size=(1000, 1))  # 1000 binary labels (0 or 1)

#  Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5048 - loss: 0.6986
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5139 - loss: 0.6976
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5709 - loss: 0.6882
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5334 - loss: 0.6883
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5530 - loss: 0.6873
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5792 - loss: 0.6833
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5780 - loss: 0.6787
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5575 - loss: 0.6761
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5511 - loss: 0.6818
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6069 - loss: 0.6746


In the above code:

`X_train` and `y_train` are placeholders for your actual training data.

`model.fit` trains the model for a specified number of epochs and batch size.


Evaluate the Model

After training, evaluate the model on test data to see how well it performs.


In [18]:
# Example test data (in practice, use real dataset)
X_test = np.random.rand(200, 20)  # 200 samples, 20 features each
y_test = np.random.randint(2, size=(200, 1))  # 200 binary labels (0 or 1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)

# Print test loss and accuracy
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4940 - loss: 0.7034  
Test loss: 0.7023367285728455
Test accuracy: 0.5099999904632568


In the above code:

`model.evaluate` computes the loss and accuracy of the model on test data.

`X_test` and `y_test` are placeholders for your actual test data.


 Basic Custom Training Loop



- Set up the environment and load the dataset.

- Define the model with a Flatten layer and two Dense layers.

- Define the loss function and optimizer.

- Implement a custom training loop to iterate over the dataset, compute the loss, and update the model's weights.


In [21]:
# Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# Define the model with a Flatten layer and two Dense layers
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

# Define the loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# Implement a custom training loop to iterate over the dataset, compute the loss, and update the model's weights
epochs = 5
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True) # forward pass
            loss_value = loss_fn(y_batch_train, logits) # compute loss

        # compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 100 stepss
        if step % 100 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')

Start of epoch 1
Epoch 1 Step 0: Loss = 2.3399946689605713
Epoch 1 Step 100: Loss = 0.581192135810852
Epoch 1 Step 200: Loss = 0.35511648654937744
Epoch 1 Step 300: Loss = 0.25517189502716064
Epoch 1 Step 400: Loss = 0.17661848664283752
Epoch 1 Step 500: Loss = 0.3493261933326721
Epoch 1 Step 600: Loss = 0.16815727949142456
Epoch 1 Step 700: Loss = 0.1142866462469101
Epoch 1 Step 800: Loss = 0.15714596211910248
Epoch 1 Step 900: Loss = 0.10838490724563599
Epoch 1 Step 1000: Loss = 0.4504319429397583
Epoch 1 Step 1100: Loss = 0.16053643822669983
Epoch 1 Step 1200: Loss = 0.16539366543293
Epoch 1 Step 1300: Loss = 0.17619968950748444
Epoch 1 Step 1400: Loss = 0.2547246515750885
Epoch 1 Step 1500: Loss = 0.1651730239391327
Epoch 1 Step 1600: Loss = 0.21836332976818085
Epoch 1 Step 1700: Loss = 0.08938762545585632
Epoch 1 Step 1800: Loss = 0.16124871373176575
Start of epoch 2
Epoch 2 Step 0: Loss = 0.06793273985385895
Epoch 2 Step 100: Loss = 0.21987612545490265
Epoch 2 Step 200: Loss = 0.

Adding Accuracy Metric <br/>

Enhance the custom training loop by adding an accuracy metric to monitor model performance.


1. Set up the environment and define the model, loss function, and optimizer.

2. Add Sparse Categorical Accuracy as a metric.

3. Implement the custom training loop with accuracy tracking.


In [23]:
# Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# Define the model with a Flatten layer and two Dense layers
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

# Define Loss Function, Optimizer, and Metric
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training

# Implement the custom training loop with accuracy
epochs = 10

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
          logits = model(x_batch_train, training=True) # forward pass: compute predictions
          loss_value = loss_fn(y_batch_train, logits) # compute loss

        # compute grandients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        # update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 100 steps
        if step % 100 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')

    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()

Start of epoch 1
Epoch 1 Step 0: Loss = 2.4485409259796143 Accuracy = 0.09375
Epoch 1 Step 100: Loss = 0.624505877494812 Accuracy = 0.7667078971862793
Epoch 1 Step 200: Loss = 0.41013437509536743 Accuracy = 0.8334888219833374
Epoch 1 Step 300: Loss = 0.2480856031179428 Accuracy = 0.8539243936538696
Epoch 1 Step 400: Loss = 0.18354114890098572 Accuracy = 0.8673628568649292
Epoch 1 Step 500: Loss = 0.44643551111221313 Accuracy = 0.8733158707618713
Epoch 1 Step 600: Loss = 0.18937981128692627 Accuracy = 0.8829554915428162
Epoch 1 Step 700: Loss = 0.11431771516799927 Accuracy = 0.890647292137146
Epoch 1 Step 800: Loss = 0.16023290157318115 Accuracy = 0.8957163095474243
Epoch 1 Step 900: Loss = 0.11517751216888428 Accuracy = 0.9003537893295288
Epoch 1 Step 1000: Loss = 0.44569969177246094 Accuracy = 0.9030344486236572
Epoch 1 Step 1100: Loss = 0.22025974094867706 Accuracy = 0.9065905809402466
Epoch 1 Step 1200: Loss = 0.19484612345695496 Accuracy = 0.909268319606781
Epoch 1 Step 1300: Loss 

Custom Callback for Advanced Logging <br/>

 Implement a custom callback to log additional metrics and information during training.


1. Set up the environment and define the model, loss function, optimizer, and metric.

2. Create a custom callback to log additional metrics at the end of each epoch.

3. Implement the custom training loop with the custom callback.


In [25]:
# Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# Define the model with a Flatten layer and two Dense layers
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

# Define Loss Function, Optimizer, and Metric
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training

# Implement the custom training loop with accuracy
epochs = 2
custom_callback = CustomCallback()

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
          # Forward pass
          logits = model(x_batch_train, training=True)
          # Compute loss
          loss_value = loss_fn(y_batch_train, logits)

        # compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')

    #  Call the custom callback at the end of each step
    custom_callback.on_epoch_end(epoch, logs={'loss': loss_value.numpy(), 'accuracy': accuracy_metric.result().numpy()})

    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()

Start of epoch 1
Epoch 1 Step 0: Loss = 2.4378397464752197 Accuracy = 0.125
Epoch 1 Step 200: Loss = 0.39522427320480347 Accuracy = 0.8260261416435242
Epoch 1 Step 400: Loss = 0.1886368691921234 Accuracy = 0.8631545901298523
Epoch 1 Step 600: Loss = 0.17917487025260925 Accuracy = 0.8811355829238892
Epoch 1 Step 800: Loss = 0.15405292809009552 Accuracy = 0.89454585313797
Epoch 1 Step 1000: Loss = 0.4464195966720581 Accuracy = 0.9023476243019104
Epoch 1 Step 1200: Loss = 0.19408738613128662 Accuracy = 0.9090341329574585
Epoch 1 Step 1400: Loss = 0.25699207186698914 Accuracy = 0.9139900207519531
Epoch 1 Step 1600: Loss = 0.23609337210655212 Accuracy = 0.9176881909370422
Epoch 1 Step 1800: Loss = 0.13588860630989075 Accuracy = 0.9218316078186035
End of epoch 1, loss: 0.041016072034835815, accuracy: 0.9237333536148071
Start of epoch 2
Epoch 2 Step 0: Loss = 0.0747869610786438 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.14780940115451813 Accuracy = 0.9626865386962891
Epoch 2 Step 400: Loss = 0

Hyperparameter Tuning <br/>

Add functionality to save the results of each hyperparameter tuning iteration as JSON files in a specified directory.


Modify the tuning loop to save each iteration's results as JSON files.

Specify the directory where these JSON files will be stored for easier retrieval and analysis of tuning results.


In [27]:
!pip install keras-tuner
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [32]:
import json
import os
import keras_tuner as kt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Load your dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Define the model-building function
def build_model(hp):
    model = Sequential()
    # Tune the number of units in the first Dense layer
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification example
    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize a Keras Tuner RandomSearch tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Set the number of trials
    executions_per_trial=1,  # Set how many executions per trial
    directory='tuner_results',  # Directory for saving logs
    project_name='hyperparam_tuning'
)

# Run the tuner search (make sure the data is correct)
tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=5)

# Create the directory for saving results if it doesn't exist
results_dir = 'tuning_results_json' # Using a different directory name to avoid conflict with tuner's directory
os.makedirs(results_dir, exist_ok=True)

# Save the tuning results as JSON files
try:
    for i in range(10):
        # Fetch the best hyperparameters from the tuner
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

        # Results dictionary to save hyperparameters and score
        results = {
            "trial": i + 1,
            "hyperparameters": best_hps.values,  # Hyperparameters tuned in this trial
            "score": None  # Add any score or metrics if available
        }

        # Save the results as JSON
        with open(os.path.join('tuning_results_json', f"trial_{i + 1}.json"), "w") as f:
            json.dump(results, f)

except IndexError:
    print("Tuning process has not completed or no results available.")

Reloading Tuner from tuner_results/hyperparam_tuning/tuner0.json


Explanation of Hyperparameter Tuning <br/>




Explanation: "num_trials specifies the number of top hyperparameter sets to return. Setting num_trials=1 means that it will return only the best set of hyperparameters found during the tuning process."